In [6]:
# ML model for AquaPolis 
#based on dataset with water parameters from sensors as input and the quality of the water as output (attributes, classification problem)

#get dataset from local drive
import pandas as pd
import requests

# url = 'https://github.com/mp619/Aqua-Polis/blob/master/models/water_potability.csv'
# requests.get(url)
# df1 = pd.read_csv(url)
# # Dataset is now stored in a Pandas Dataframe

from google.colab import files
uploaded = files.upload()

import io
df2 = pd.read_csv(io.BytesIO(uploaded['water_potability.csv']))


Saving water_potability.csv to water_potability (1).csv


In [64]:
#get in and outp
print(df2)
print(type(df2))

#reformat data
import sklearn
from sklearn.impute import SimpleImputer
cols = df2.columns
imp_mean = SimpleImputer()
data_trans = imp_mean.fit_transform(df2)
data_trans = pd.DataFrame(data_trans, columns = cols)
data_trans.head()
data_shuffled = data_trans.sample(frac=1).reset_index(drop=True)

print(data_shuffled)
data = data_shuffled.iloc[:, [0,1,2,3,4,5,6,7,8]]
labels = data_shuffled.iloc[:, [9]]


print(data)
print(labels)

            ph    Hardness  ...  Turbidity  Potability
0          NaN  204.890455  ...   2.963135           0
1     3.716080  129.422921  ...   4.500656           0
2     8.099124  224.236259  ...   3.055934           0
3     8.316766  214.373394  ...   4.628771           0
4     9.092223  181.101509  ...   4.075075           0
...        ...         ...  ...        ...         ...
3271  4.668102  193.681735  ...   4.435821           1
3272  7.808856  193.553212  ...   2.798243           1
3273  9.419510  175.762646  ...   3.298875           1
3274  5.126763  230.603758  ...   4.708658           1
3275  7.874671  195.102299  ...   2.309149           1

[3276 rows x 10 columns]
<class 'pandas.core.frame.DataFrame'>
             ph    Hardness  ...  Turbidity  Potability
0      6.124672  179.579752  ...   3.631007         0.0
1      5.853561  186.807751  ...   3.756874         0.0
2      7.080795  190.267367  ...   3.796737         1.0
3      7.397413  122.541040  ...   3.149158         

In [65]:
#data manipulation
from dataclasses import dataclass
import cv2
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import seed

#resize??

seed(1234)
data = np.asarray(data).astype('float32')
labels = np.asarray(labels).astype('float32')

print(labels.shape)
#split in training and validation set
split_size_val = int(0.8*len(data))
X_train, X_val = data[:split_size_val], data[split_size_val+1:]
Y_train, Y_val = labels[:split_size_val], labels[split_size_val+1:]

X_train = X_train.astype('float32')
X_val = X_val.astype('float32')
Y_train = Y_train.astype('float32')
Y_val = Y_val.astype('float32')
# # Normalize the data
# X_train /= 255
# X_val /= 255

# Print shape of training and val images 
print('X_train image shape: {0}'.format(X_train.shape))
print('X_val image shape: {0}'.format(X_val.shape))

# Print shape of training and val labels 
print('Y_train labels shape: {0}'.format(Y_train.shape))
print('Y_val labels shape: {0}'.format(Y_val.shape))

(3276, 1)
X_train image shape: (2620, 9)
X_val image shape: (655, 9)
Y_train labels shape: (2620, 1)
Y_val labels shape: (655, 1)


In [73]:
# model 

import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.utils import np_utils
from keras.datasets import cifar10
from tensorflow.keras.optimizers import RMSprop, Adam
from keras.layers import Conv2D, MaxPooling2D, Flatten, BatchNormalization, Dropout
from keras import regularizers

pdrop = 0.2

model = Sequential()
model.add(Dense(10, kernel_initializer='random_normal',
                bias_initializer='zeros', input_dim=9))
model.add(Activation('sigmoid'))
model.add(BatchNormalization())
model.add(Dense(10))
model.add(Activation('sigmoid'))
model.add(Dropout(pdrop))
model.add(Dense(1))
model.add(Activation('softmax'))

model.summary()

# initiate RMSprop optimizer
opt = Adam(lr=1e-4)

model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
history = model.fit(X_train, Y_train, batch_size=32, epochs=20)
score = model.evaluate(X_val, Y_val, verbose=0)
print(score)
print('Validation loss:', score[0])
print('Validation accuracy:', score[1])

Model: "sequential_32"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_81 (Dense)            (None, 10)                100       
                                                                 
 activation_74 (Activation)  (None, 10)                0         
                                                                 
 batch_normalization_5 (Batc  (None, 10)               40        
 hNormalization)                                                 
                                                                 
 dense_82 (Dense)            (None, 10)                110       
                                                                 
 activation_75 (Activation)  (None, 10)                0         
                                                                 
 dropout_17 (Dropout)        (None, 10)                0         
                                                     

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


82/82 [==============================] - 1s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3973
Epoch 2/20
82/82 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3973
Epoch 3/20
82/82 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3973
Epoch 4/20
82/82 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3973
Epoch 5/20
82/82 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3973
Epoch 6/20
82/82 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3973
Epoch 7/20
82/82 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3973
Epoch 8/20
82/82 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3973
Epoch 9/20
82/82 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3973
Epoch 10/20
82/82 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accurac

In [69]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification
clf = AdaBoostClassifier(n_estimators=100,  random_state=0)
print(X_train)
print(Y_train)
clf.fit(X_train, Y_train)
clf.predict(X_val)
# mean accuracy 
clf.score(X_val,Y_val)

#gridsearch form sk learn to chose best hyperparameters


[[6.1246724e+00 1.7957976e+02 2.7951613e+04 ... 1.3835116e+01
  6.9017487e+01 3.6310067e+00]
 [5.8535604e+00 1.8680775e+02 1.9790857e+04 ... 1.9670702e+01
  6.5040009e+01 3.7568738e+00]
 [7.0807943e+00 1.9026736e+02 1.5255115e+04 ... 1.3312723e+01
  6.8944885e+01 3.7967374e+00]
 ...
 [5.5242968e+00 1.5714243e+02 2.6220174e+04 ... 1.6903181e+01
  5.1106613e+01 4.0333910e+00]
 [5.1743460e+00 2.3432857e+02 1.1746607e+04 ... 1.3771680e+01
  3.6183537e+01 3.6891451e+00]
 [6.5726562e+00 1.9264734e+02 1.4598952e+04 ... 1.7343805e+01
  5.3085480e+01 2.8209271e+00]]
[[0.]
 [0.]
 [1.]
 ...
 [0.]
 [0.]
 [1.]]


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.616793893129771